In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('../../Data/heart_2022_with_nans.csv')

In [ ]:
# prepare data for logistic regression
def logi_reg(df,target, target_names):
    """You know what it does 

        Input: 
            df: pandas dataframe
            target: target column name
            target_names: the names of the target classes eg."Yes" and "No"

        Output:
            None 
    """
    df_train, df_test = train_test_split(df, 
                                        test_size = 0.25, 
                                        random_state = 69,
                                        stratify = df[target])

    X_train = df_train.drop(columns = target)
    y_train = df_train[target]
    X_test = df_test.drop(columns = target)
    y_test = df_test[target]

    # print the size of the training and test set
    print (f"Size of training set : {df_train.shape[0]} rows , {df_train.shape[1]} columns")
    print (f"Size of training set : {df_test.shape[0]} rows , {df_test.shape[1]} columns")

    # fit logistic regression model with elastic net regularization
    log_reg = LogisticRegression(penalty = 'elasticnet',
                                solver = 'saga',
                                l1_ratio = 0.5,
                                max_iter = 1000)
    
    log_reg.fit(X_train, y_train)

    # predict on test set
    y_test_pred = log_reg.predict(X_test)
    y_pred_proba = log_reg.predict_proba(X_test)[:,1]

    # print metrics
    print('\n===============================\n'+
      'Classification report on test data' +
      '\n===============================\n')
    print(classification_report(y_test, y_test_pred, target_names=target_names))

    print('\n===============================\n'+
        'Confusion matrix on test data' +
        '\n===============================\n')
    print(confusion_matrix(y_test, y_test_pred))

